In [21]:
import tweepy
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API 

from textblob import TextBlob
 
import twitter_credentials

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

import pandas as pd
from pandas import DataFrame

In [36]:
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = OAuthHandler(twitter_credentials.consumer_key, twitter_credentials.consumer_secret)
        auth.set_access_token(twitter_credentials.access_token, twitter_credentials.access_token_secret)
        return auth

class TwitterClient(object):
    
    # We want to create a generic twitter class for sentiment analysis
    
    def __init__(self):
  
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(twitter_credentials.consumer_key, twitter_credentials.consumer_secret)
            # set access token and secret
            self.auth.set_access_token(twitter_credentials.access_token, twitter_credentials.access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
            self.twitterclient = API(self.auth)
        except:
            print("Error: Authentication Failed")
            
    def get_twitter_client_api(self):
        return self.twitterclient
    
    def clean_tweet(self, tweet):
        
        # Utility function to clean tweet text by removing links, special characters
        # using simple regex statements.
        
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
  
    def get_tweet_sentiment(self, tweet):
        
        # Utility function to classify sentiment of passed tweet
        # using textblob's sentiment method
        
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
  
    def get_tweets(self, query, count = 20):
        
        # Main function to fetch tweets and parse them.
        
        # empty list to store parsed tweets
        tweets = []
  
        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)
  
            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
  
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
                parsed_tweet['date'] = tweet.created_at
                parsed_tweet['id'] = tweet.id
                parsed_tweet['likes'] = tweet.favorite_count
                parsed_tweet['retweets'] = tweet.retweet_count
                
                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
  
            # return parsed tweets
            return tweets
  
        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))

class TweetAnalyzer():
    
    # Functionality for analyzing and categorizing content from tweets.
    
    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

        df['id'] = np.array([tweet.id for tweet in tweets])
        df['len'] = np.array([len(tweet.text) for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])
        
        return df

def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    # calling function to get tweets
    tweets = api.get_tweets(query = 'ethereum', count = 25)
  
    # picking positive tweets from tweets
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    # percentage of positive tweets
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
    # picking negative tweets from tweets
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    # percentage of negative tweets
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
    # percentage of neutral tweets
    print("Neutral tweets percentage: {} % \
        ".format(100*(len(tweets) -(len( ntweets )+len( ptweets)))/len(tweets)))
  
    # printing first 5 positive tweets
    print("\n\nPositive tweets:")
    for tweet in ptweets[:10]:
        print(tweet['text'])
  
    # printing first 5 negative tweets
    print("\n\nNegative tweets:")
    for tweet in ntweets[:10]:
        print(tweet['text'])
        
    tweets_dataframe = pd.DataFrame(tweets)
 
    tweets_dataframe.to_csv(r'sentiment_analysis_second_draft.csv', index = False)
  
if __name__ == "__main__":
    # calling main function
    main()

Positive tweets percentage: 36.0 %
Negative tweets percentage: 24.0 %
Neutral tweets percentage: 40.0 %         


Positive tweets:
RT @JunkoSu22993224: Cardano will flip Ethereum. 

Giving away 1 Cardano to a lucky person. Drop your #Cardano wallet address below.
RT @derrybrown: There is $7.3b worth of $ETH locked in @MakerDAO to get a loan in $DAI and you have to PAY interest.

When you move your $E…
RT @K9NFT: 💎 GIVEAWAY💎
⚡ LEGENDARY $ETH NFT⚡

🕑 Winner in 48 hours!🕒
(+ 3 smaller prize) 

❤ FOLLOW
💚 RETWEET
💬 COMMENT: #Ethereum

#NFTs #…
RT @CryptoGrandpas: 🏅500$ #BNB || 
         +
 AIRPODS 2 🎧

One lucky GRANDSON🍀

Just: 

-Follow @CryptoGrandpas  
-Rt🔁
-Tag 2 friends👴🏼…
RT @bsitoken: BSI is here to offer solutions in the travel industry🙌🏻 swipe left to read more! 

👉🏻 for more information, visit our website…
RT @ShaolinShoppe: Just dropped a new piece
BOMBSHELL
5/5 0.061Ξ
https://t.co/EEGGziiJPd #rarible #ethereum #nonfungible #digitalasset #nft…
RT @Orion__Research: Giving Away 

In [38]:
import tweepy

auth = tweepy.AppAuthHandler(twitter_credentials.consumer_key, twitter_credentials.consumer_secret)
api = tweepy.API(auth)

for status in tweepy.Cursor(api.search_tweets, "bitcoin",
                            count=100).items(250):
    print(status.id)

AttributeError: 'API' object has no attribute 'search_tweets'